In [ ]:
import os

In [ ]:
# Specify the new directory path
parent_directory = 'c:/Zhou/Ma/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
# import csv

1. read yield data and filled img 

In [ ]:
from dataset import select_data_and_yield_list
from dl.dl_prediction import select_imglist_yield
from ml.ml_predict import ml_predict_yield, prepare_train_test_data

In [ ]:
from config_constant import crop_var, irrigate_var, CROP_TYPE
from plot_utils import plot_distinct_yields

In [ ]:

yield_file = 'C:/Users/yutzhou/Desktop/Corn_Yield/BL2022_Yld_label.csv'
weather_file = 'C:/Users/yutzhou/Desktop/Corn_Yield/CoAgMet ET_VBA_calc_2022.xlsm'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
# out_path = '../output/DOY201/'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
# out_path = '../output/DOY269/'
# doy=269
img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220916_DOY259_extracted_filled'
out_path = '../output/DOY259/'
doy = 259
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220716_DOY197_extracted_filled'
# out_path = '../output/DOY197/'
# doy = 197
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220708_DOY189_extracted_filled'
# out_path = '../output/DOY189/'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220705_DOY186_extracted_filled'
# out_path = '../output/DOY186/'
# doy = 186
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled'
# out_path = '../output/DOY179/'
# doy=179
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled'
# out_path = '../output/DOY167/'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220610_DOY161_extracted_filled'
# out_path = '../output/DOY161/'
if not os.path.exists(out_path):
    # If it doesn't exist, create the directory and any missing parent directories
    os.makedirs(out_path)
    print(f"Directory '{out_path}' created successfully.")
else:
    print(f"Directory '{out_path}' already exists.")

name_tag = img_path[-23:-17]

In [ ]:
suf_name ={'LWIR': 'Thermal'}

1.0. Image only

In [ ]:

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']

# key_word_list = ['Ref_filled.tif']
# suffix_list_list = [[], ['LWIR_filled.tif']]
key_word_list = ['RGB_filled2.tif']
suffix_list_list = [[], ['LWIR_filled2.tif']]
# suffix_list_list = [[]]
# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
# VI_list = ['ndvi',  'evi']
# VI_list = ['ndvi',  'evi', 'ndre', 'gndvi']
VI_list = []

vi_only = False

analyze_variety_id=2
analyze_irrigation_id=2
train_col='TRAIN_75'

for keyword in key_word_list:    
    
    img_list, yield_pf = select_imglist_yield(yield_file, img_path, keyword, 
                                            analyze_variety_id=analyze_variety_id, analyze_irrigation_id=analyze_irrigation_id)
    
    
    for suffix_list in suffix_list_list:
        train_images, test_images, train_yields, test_yields= prepare_train_test_data(img_list, yield_pf,train_col=train_col, 
                            VI_list=VI_list, suffix_list=suffix_list, vi_only=False)
        
        
        # test_modes = ['LR']
        test_modes = ['Lasso', 'LR', 'GB', 'RF', 'XGB']
        # test_modes = ['Lasso', 'LR', 'RF']
        # test_modes = ['Lasso', 'LR', 'SVR', 'GB']
        for modelname in test_modes:
            
            out_name = name_tag + ' ' + CROP_TYPE[crop_var[analyze_variety_id]] + ' ' 
            if not vi_only:
                out_name = out_name + keyword[:3]
            for vi_name in VI_list:
                out_name = out_name + '+' + vi_name.upper()
            for suf in suffix_list:
                out_name = out_name + '+' + suf_name[suf[:suf.rfind('_')]]
            out_name = out_name + ' '
            
            cur_time = time.time()
            trained_model = ml_predict_yield(train_images, train_yields, test_images, modelname, out_name, out_path)
            
            training_time = time.time() - cur_time
            print('training time is : ', training_time)
            
            
            cur_time = time.time()
            pred_yields = trained_model.predict(test_images)
            predict_time = time.time() - cur_time
            print('predict time is : ', predict_time)
            
            test_irrigate_data = np.array(yield_pf[yield_pf[train_col] == 0]['Irrigation_int'])
            test_variety_data = np.array(yield_pf[yield_pf[train_col] == 0]['Variety_int'])

            title = out_name +'('+ modelname +') '
            out_name = out_name + modelname +' '


            result_df=pd.DataFrame({
                'Truth': np.array(test_yields), 
                'Prediction':np.array(pred_yields),
                'Irrigation_int':test_irrigate_data,
                'Vriaty_int':test_variety_data
            })
            csv_file_path = out_path+out_name + '.csv'
            result_df.to_csv(csv_file_path, index=False)
            
            
            test_yields = np.array(test_yields)
            pred_yields = np.array(pred_yields)
            plot_distinct_yields(np.array(test_yields), np.array(pred_yields), test_irrigate_data, test_variety_data, title, out_path+out_name)
                        
#           
            if analyze_variety_id >2:
                for var_id in set(test_variety_data):
                    
                    subtitle = title + str(var_id)
                    sub_test_yields = test_yields[test_variety_data == var_id]
                    sub_pred_yields = pred_yields[test_variety_data == var_id]
                    sub_test_irrigate_data = test_irrigate_data[test_variety_data == var_id]
                    sub_test_variety_data = test_variety_data[test_variety_data == var_id]
                    plot_distinct_yields(sub_test_yields, sub_pred_yields, 
                                            sub_test_irrigate_data, sub_test_variety_data, subtitle, out_path+subtitle)
    #a

2. with Metadata

In [ ]:
from dataset import create_metadata

In [ ]:

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['RGB_filled2.tif']
suffix_list_list = [[], ['LWIR_filled2.tif']]
# suffix_list_list = [[]]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
# VI_list = ['ndvi']
# VI_list = ['ndvi',  'evi', 'ndre', 'gndvi']
VI_list = []

vi_only = False

analyze_variety_id=2
analyze_irrigation_id=2
train_col='TRAIN_75'
is_meta = True

for keyword in key_word_list:    
    
    img_list, yield_pf = select_imglist_yield(yield_file, img_path, keyword, 
                                            analyze_variety_id=analyze_variety_id, analyze_irrigation_id=2)
    
    if analyze_variety_id != 3 or analyze_irrigation_id != 2:
        if analyze_variety_id != 3 and analyze_irrigation_id != 2:
            yield_pf = yield_pf[(yield_pf['Variety_int'] == analyze_variety_id) & (yield_pf['Irrigation_int'] == analyze_irrigation_id)]
        elif analyze_variety_id != 3:
            yield_pf = yield_pf[yield_pf['Variety_int'] == analyze_variety_id]
        elif  analyze_irrigation_id != 2:
            yield_pf = yield_pf[ yield_pf['Irrigation_int'] == analyze_irrigation_id]
            
        indices = yield_pf.index.tolist()
        img_list = [img_list[i] for i in indices]
        #### yield_pf = yield_pf[yield_pf['Variety_int'] == analyze_variety_id & yield_pf['Irrigation_int'] == analyze_irrigation_id]
        yield_pf = yield_pf.reset_index(drop=True)

    # yield_list = list(yield_pf['Yield_Bu_Ac'])
    yield_list = list(yield_pf['Yield_MT_Ha'])

    '''get metadata'''
    metadata = create_metadata(yield_pf, weather_file,doy)
    
    metadata = metadata.values.tolist()
    print(len(metadata))
    
    for suffix_list in suffix_list_list:
        train_images, test_images, train_yields, test_yields= prepare_train_test_data(img_list, yield_pf,metadata=metadata,
                            train_col=train_col, VI_list=VI_list, suffix_list=suffix_list, vi_only=False)
        
        
        test_modes = ['Lasso', 'LR','XGB','RF', 'GB']
        # test_modes = ['XGB']
        # test_modes = ['Lasso', 'LR']
        # test_modes = ['Lasso', 'LR', 'SVR', 'GB']
        for modelname in test_modes:
            
            out_name = name_tag + ' ' + CROP_TYPE[crop_var[analyze_variety_id]] + ' ' 
            if not vi_only:
                out_name = out_name + keyword[:3]
            for vi_name in VI_list:
                out_name = out_name + '+' + vi_name.upper()
            for suf in suffix_list:
                out_name = out_name + '+' + suf[:suf.rfind('_')]
            out_name = out_name + ' '
            
            cur_time = time.time()
            
            trained_model = ml_predict_yield(train_images, train_yields, test_images, modelname, out_name, out_path, is_meta)
            
            training_time = time.time() - cur_time
            print('training time is : ', training_time)
            
            
            cur_time = time.time()
            pred_yields = trained_model.predict(test_images)
            predict_time = time.time() - cur_time
            print('predict time is : ', predict_time)
            
            test_irrigate_data = np.array(yield_pf[yield_pf[train_col] == 0]['Irrigation_int'])
            test_variety_data = np.array(yield_pf[yield_pf[train_col] == 0]['Variety_int'])

            title = out_name + modelname +' '
            out_name = out_name + modelname +' '


            result_df=pd.DataFrame({
                'Truth': np.array(test_yields), 
                'Prediction':np.array(pred_yields),
                'Irrigation_int':test_irrigate_data,
                'Vriaty_int':test_variety_data
            })
            csv_file_path = out_path+out_name + '.csv'
            result_df.to_csv(csv_file_path, index=False)

            test_yields = np.array(test_yields)
            pred_yields = np.array(pred_yields)
            plot_distinct_yields(test_yields, pred_yields, test_irrigate_data, test_variety_data, title, out_path+out_name)
            
            
            if analyze_variety_id >2:
                for var_id in set(test_variety_data):
                    
                    subtitle = title + str(var_id)
                    sub_test_yields = test_yields[test_variety_data == var_id]
                    sub_pred_yields = pred_yields[test_variety_data == var_id]
                    sub_test_irrigate_data = test_irrigate_data[test_variety_data == var_id]
                    sub_test_variety_data = test_variety_data[test_variety_data == var_id]
                    plot_distinct_yields(sub_test_yields, sub_pred_yields, 
                                            sub_test_irrigate_data, sub_test_variety_data, subtitle, out_path+subtitle)
            

In [ ]:
test_yields[test_variety_data == 2]

3. Generalize

In [ ]:
img_path_list=[
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220616_DOY167_extracted_filled',
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220628_DOY179_extracted_filled',
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220708_DOY189_extracted_filled',
    'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
]
doy_list=[167, 179, 189, 201]

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
image_list = [np.random.rand(64, 64, 3) for _ in range(3000)]  # Replace with your actual list of images

# Generate random target variable for illustration purposes (replace this with your actual target)
target_variable = np.random.rand(3000)

# Flatten each image in the list
flattened_images = [image.flatten() for image in image_list]

# Convert the list of flattened images to a 2D NumPy array
X_flattened = np.array(flattened_images)

print(X_flattened.shape)

X_train, X_test, y_train, y_test = train_test_split(X_flattened, target_variable, test_size=0.2, random_state=42)
print(X_train.shape)
# Create and fit a Lasso regression model
lasso_model = Lasso(alpha=0.1)  # Adjust the alpha parameter as needed
lasso_model.fit(a, train_yields)

In [ ]:
print(train_images[0].shape)

In [ ]:
print(len(train_images[0][0]), len(train_images[1][0]))
array_1 = train_images[0].reshape((train_images[0].shape[0], -1))
array_2 = train_images[1]
a = np.concatenate((array_1, np.array(array_2)),axis=1)
print(a.shape)

In [ ]:
lasso_model.fit(a, train_yields)


## NDVI only 

In [ ]:
import numpy as np

In [ ]:
image_shape = (5, 64, 64)  # Example: 64x64 image
num_samples = 1000

# Synthetic 2D image data
X_images = np.random.rand(num_samples, *image_shape)

# Synthetic 1D CSV data
X_csv = np.random.rand(num_samples, 10)

X_train = [X_images, X_csv]

In [ ]:
print(X_train)

In [ ]:

# deficit_indices = list(yield_pf[yield_pf[train_col] == 1].index)\

### 

In [ ]:

# yield_file = 'C:/Users/yutzhou/Desktop/Corn_Yield/BL2022_Yld.csv'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
# out_path = 'output/'

# name_tag = img_path[-23:-17]
# # print(name_tag)

# selection = 'Pioneer' # 
# # selection = 'Pioneer Deficit' 
# # selection = 'Pioneer Full'
# key_word_list = ['Ref_filled.tif']
# suffix_list_list = [None, ['LWIR_filled.tif']]

# # suffix = ['base', 'lwir']
# # suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi']
# for keyword in key_word_list: 
#     pioneer_img_list, pioneer_yield_list = select_data_and_yield_list(img_path, yield_file, key_word = keyword, selection = selection)

#     for suffix_list in suffix_list_list:

        
#         pioneer_dataset = get_ml_image(pioneer_img_list, VI_list = VI_list, suffix_list = suffix_list)
#         print(pioneer_dataset.shape
#                 )
#         pioneer_yield = np.array(pioneer_yield_list)
        
#         test_modes = ['Lasso', 'LR', 'RF']
#         # test_modes = ['Lasso']
#         for modelname in test_modes:
            
#             # out_name = name_tag + ' ' + selection + ' ' + keyword[:-11] + ' NDVI ' 
#             out_name = name_tag + ' ' + selection + ' NDVI '  
#             if suffix_list is not None:
#                 for suf in suffix_list:
#                     out_name = out_name + suf[:-11]
#             out_name = out_name + ' '
#             ml_predict_yield(pioneer_dataset, pioneer_yield, modelname, out_name, out_path)

### load models

In [ ]:
from sklearn.model_selection import train_test_split
from ml.ml_predict import test_model

import pickle
model_name = 'output\DOY197\DOY197 Pioneer +NDVI RF.pkl'
# Load the model from the .pkl file using pickle
with open(model_name, 'rb') as model_file:
    model = pickle.load(model_file)
    
    
vi_only = True    
yield_file = 'C:/Users/yutzhou/Desktop/Corn_Yield/BL2022_Yld.csv'
img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220720_DOY201_extracted_filled'
out_path = 'output/LIRF20220720'

name_tag = img_path[-23:-17]
# print(name_tag)

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['Ref_filled.tif']
suffix_list_list = [[], ['LWIR_filled.tif']]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
VI_list = ['ndvi']


selection = ['Pioneer'] # 
# selection = 'Pioneer Deficit' 
# selection = 'Pioneer Full'
pioneer_img_list, pioneer_yield_list, irrigate_type_list = select_data_and_yield_list(
    img_path, yield_file, key_word = 'Ref_filled.tif', crop_type_select=selection)


dataset = get_ml_image(pioneer_img_list, VI_list=VI_list, suffix_list = [], is_vi_only=vi_only)

# break
yield_data = np.array(pioneer_yield_list)
irrigate_data = np.array(irrigate_type_list)

    
original_indices = np.arange(len(yield_data))
train_images, test_images, train_yields, test_yields, train_indices, test_indices = train_test_split(
dataset, yield_data, original_indices, test_size=0.2, random_state=39)

test_model(model, test_images, test_yields, test_indices, irrigate_data, 'output/apply_model_test')



In [ ]:
# final_estimator = model.named_steps['classifier']
# # Access feature importances
# feature_importances = final_estimator.feature_importances_

In [ ]:
print(dataset.shape)

### get 



In [ ]:
# from sklearn.model_selection import train_test_split
# yield_file = 'C:/Users/yutzhou/Desktop/Corn_Yield/BL2022_Yld.csv'
# img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
# out_path = 'output/'

# name_tag = img_path[-23:-17]
# # print(name_tag)

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
# suffix_list_list = [None, ['LWIR_filled.tif']]

# # suffix = ['base', 'lwir']
# # suffix_list = ['LWIR_filled.tif']
# VI_list = []

# selection = 'Pioneer' # 
# # selection = 'Pioneer Deficit' 
# # selection = 'Pioneer Full'
# pioneer_img_list, pioneer_yield_list, pioneer_type_list = select_data_and_yield_list(
# img_path, yield_file, key_word = 'Ref_filled.tif', selection = selection)

# print(pioneer_img_list[0])
# print(pioneer_img_list[100])
# print(pioneer_img_list[1000])
# print(pioneer_img_list[2000])
# print(pioneer_img_list[3000])

# pioneer_dataset = get_ml_image(pioneer_img_list, VI_list=VI_list, suffix_list = [])
# pioneer_yield = np.array(pioneer_yield_list)
# pioneer_type = np.array(pioneer_type_list)
# index_pivot = 1740

# # 
# original_indices = np.arange(len(pioneer_yield))
# train_images, test_images, train_yields, test_yields, train_indices, test_indices = train_test_split(
# pioneer_dataset, pioneer_yield, original_indices, test_size=0.2, random_state=39)
# index_1 = test_yields[np.where(test_indices<index_pivot)]
# index_2 = test_yields[np.where(test_indices>=index_pivot)]

Draw plots from result csv

In [ ]:
# from path_utils import get_files_by_suffix
# from ml.ml_predict import plot_result, plot_result_separate
# fig_path = './output/Ref+NDVI'
# suffixe = 'csv'
# files = get_files_by_suffix(fig_path, suffixe)

# for file in files:
#     data = np.loadtxt(file)
#     savename = file[:-4]
    
#     # plot_result(data[:,0], data[:,1], savename)
    
#     plot_result_separate(data[:,0], data[:,1],test_indices, 1740, savename)
#     # break



# Gridsearch

In [ ]:
# param_grid = {
#     'n_estimators': [100, 150, 200],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }
# MEAN = np.nanmean(train_images,(0,2,3))
# STD = np.nanstd(train_images, (0,2,3))
# train_images = (train_images- MEAN[None,:,None,None])/STD[None,:,None,None] 
# train_images = train_images.reshape((train_images.shape[0], -1))
# rf_model = RandomForestRegressor(random_state=42)

# # Create GridSearchCV instance with the model, parameter grid, and cross-validation
# grid_search = GridSearchCV(estimator=rf_model
# , param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# # Perform the grid search on the dataset
# grid_search.fit(train_images, train_yields)

# # Print the best parameters and corresponding mean squared error
# print("Best Parameters:", grid_search.best_params_)
# print("Best Mean Squared Error:", -grid_search.best_score_)

In [ ]:
# # Initiaze the hyperparameters for each dictionary
# param1 = {}
# param1['classifier'] = [LinearRegression()]

# param2 = {}
# param2['classifier__alpha'] = [0.1, 0.3, 0.5, 1]
# param2['classifier'] = [Ridge()]

# param3 = {}
# param3['classifier__alpha'] = [0.2, 0.3, 0.4, 0.5, 0.6]
# param3['classifier'] = [Lasso()]

# param4 = {}
# param4['classifier__n_neighbors'] = [2,5,10,25,50]
# param4['classifier'] = [KNeighborsRegressor()]

# """
# param5 = {}
# param5['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
# param5['classifier__epsilon'] = [0.0,0.2,0.5,1]
# param5['classifier'] = [LinearSVR()]
# """

In [ ]:
# pipeline = pipe = Pipeline(steps=[("scaler", CustomScaler(MEAN,STD)),
#                        ("flatten", FlattenTransformer()),
#                        ("classifier", LinearRegression())])
# params = [param1, param2, param3, param4] # param5
# params = [param1, param2, param3, param4] # param5


In [ ]:
# # Train the grid search model
# grid_search = GridSearchCV(pipeline, params, cv=3, scoring='neg_mean_squared_error').fit(train_images,train_yields)

In [ ]:
# # Best performing model and its corresponding hyperparameters
# grid_search.best_params_

In [ ]:
# model = grid_search.best_estimator_
# # predict
# pred = model.predict(train_images)

# # validate mse error
# mse = mean_squared_error(train_yields, pred) 
# rmse = np.sqrt(mse)

# print(mse, rmse)

In [ ]:
# # model = grid_search.best_estimator_
# # predict
# pred = model.predict(test_images)

# # validate mse error
# mse = mean_squared_error(test_yields, pred) 
# rmse = np.sqrt(mse)

# print(mse, rmse)

In [ ]:
# save_name = 'test'
# plot_result(test_yields, pred, save_name)